In [1]:
import os
import pandas as pd

workdir = os.path.abspath('./')
print(workdir)

/Users/dongjiacheng/Desktop/code/lab/YM


In [3]:
# pd.读取excel
df_sc = pd.read_excel('./autophagy gene list-yeast-total 268 gene.xlsx')
df_sc.head()

,Gene,Ano_seq
0,YDR425W,MDYNIFEAVHEQQSSTSDMDLSEEDNNPFVGTHHLYASGIGTTIGE...
1,YJL036W,MTDKGKNDLTSKAKDKARGNPEKPPYWFEIIVSDPQKRTGDPGSSS...
2,YEL013W,MGSCCSCLKDSSDEASVSPIADNEREAVTLLLGYLEDKDQLDFYSG...
3,YHR195W,MTRPPLVRGIFSLGLSVAVLKGVEKTVRKHLERQGWIEPQKVDYEL...
4,YKR068C,MVSTTQSRSLKAMGEEIWKNKTEKINTELFTLTYGSIVAQLCQDYE...


In [5]:
"""
将df_sc中的数据，保存到一个txt中，要求格式如下：
>Protein_id + Gene + Symbol
Amin_seq
.........
"""
# 将df_sc中的数据，保存到一个txt中
with open('blast_input_Sc.txt', 'w') as f:
    for i in range(len(df_sc)):
        f.write('>' + df_sc['Gene'][i] + '\n')
        f.write(df_sc['Ano_seq'][i] + '\n')

In [6]:
# 在jupyter中添加环境变量
os.environ['PATH'] += os.pathsep + '/Users/dongjiacheng/Desktop/code/mtd/tools/blast/bin'

In [7]:
os.system('blastp -query blast_input_Sc.txt -out blast_output_Sc.txt -db /Users/dongjiacheng/Desktop/code/mtd/tools/blast/db_prot/Mt-Nc -outfmt 6 -evalue 1e-6')

0

In [8]:
# 读取blast_output_Sc.txt,
df_blast = pd.read_csv('blast_output_Sc.txt', sep='\t', header=None).copy()
df_blast.columns = ['query_id', 'subject_id', '%_Identity', 'alignment_length ', 'mismatches ', 'gap_openings', 'q_start', 'q_end', 's_start', 's_end', 'e-value', 'bit_score']

df_blast.head()

,query_id,subject_id,%_Identity,alignment_length,mismatches,gap_openings,q_start,q_end,s_start,s_end,e-value,bit_score
0,YDR425W,XP_962863.3,26.145,524,345,8,122,623,112,615,6.370000e-52,188.0
1,YDR425W,XP_003663337.1,32.869,359,212,6,122,472,97,434,5.970000e-50,182.0
2,YJL036W,XP_003658349.1,34.038,426,234,6,28,418,56,469,1.980000e-86,271.0
3,YJL036W,XP_965318.1,33.333,423,238,6,28,418,69,479,8.400000e-81,256.0
4,YJL036W,XP_011393870.1,23.810,252,146,8,75,299,444,676,1.220000e-08,56.6


In [9]:
df_blast['e-value'] = df_blast['e-value'].apply(lambda x: '{:.10f}'.format(x))
df_blast.head()

,query_id,subject_id,%_Identity,alignment_length,mismatches,gap_openings,q_start,q_end,s_start,s_end,e-value,bit_score
0,YDR425W,XP_962863.3,26.145,524,345,8,122,623,112,615,0.0000000000,188.0
1,YDR425W,XP_003663337.1,32.869,359,212,6,122,472,97,434,0.0000000000,182.0
2,YJL036W,XP_003658349.1,34.038,426,234,6,28,418,56,469,0.0000000000,271.0
3,YJL036W,XP_965318.1,33.333,423,238,6,28,418,69,479,0.0000000000,256.0
4,YJL036W,XP_011393870.1,23.810,252,146,8,75,299,444,676,0.0000000122,56.6


In [ ]:
# 

In [16]:
# 保存df_blast为excel   
df_blast.to_excel('比对结果.xlsx', index=False)